In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train.drop(['ID', 'Record Status '], axis=1, inplace=True)  # 删‘ID’列
test.drop(['ID', 'Record Status '], axis=1, inplace=True)

In [4]:
train.drop(['Secondary Address'], axis=1, inplace=True)  # 删‘Secondary Address’列
test.drop(['Secondary Address'], axis=1, inplace=True)
train.drop(['Locality'], axis=1, inplace=True)  # 删‘Locality’列
test.drop(['Locality'], axis=1, inplace=True)

In [5]:
train['Postcode'].fillna(train['Postcode'].mode()[0], inplace=True)
test['Postcode'].fillna(test['Postcode'].mode()[0], inplace=True)

train['Street'].fillna(train['Street'].mode()[0], inplace=True)
test['Street'].fillna(test['Street'].mode()[0], inplace=True)

train['District'].fillna(train['District'].mode()[0], inplace=True)
test['District'].fillna(test['District'].mode()[0], inplace=True)

In [6]:
train['Price'] = np.log1p(train['Price'])

In [7]:
train_counts = train.shape[0]
test_counts = test.shape[0]

train_Price = train['Price'][:train_counts]
train_features = train.drop(['Price'], axis=1)
test_features = test
features = pd.concat([train_features, test_features]).reset_index(drop=True)

In [8]:
features['Date '] = pd.to_datetime(features['Date '])

# 提取月、日新特征
features['Month'] = features['Date '].dt.month
features['Day'] = features['Date '].dt.day

In [9]:
# 提取'Month'特征，转换为季节信息
seasons = [(1, 'Winter'), (4, 'Spring'), (7, 'Summer'), (10, 'Fall')]
bins = [1, 4, 7, 10, 13]
labels = [name for _, name in seasons]
features['Season'] = pd.cut(features['Date '].dt.month, bins=bins, labels=labels, right=False)

# 提取'Day'特征，转换为星期几信息
features['Weekday'] = features['Date '].dt.weekday
features['Weekend'] = features['Weekday'].isin([5, 6]).astype(int)

In [10]:
# 删除原始的'Date '列
features.drop(['Date '], axis=1, inplace=True)

In [11]:
from mean_encoder import MeanEncoder

Large_Num_Features = ['Postcode', 'Street', 'First address', 'Town/City', 'District', 'County']

mean_encoder = MeanEncoder(categorical_features=Large_Num_Features)
train = mean_encoder.fit_transform(train, train['Price'])
features = mean_encoder.transform(features)

features.drop(Large_Num_Features, axis=1, inplace=True)

            Price           Date   Postcode Property Type Old/New  \
0       13.415034  2019/1/11 0:00   CM3 4BS             D       N   
1       12.206078   2019/1/4 0:00   SS3 9RJ             T       N   
2       13.208543  2019/1/18 0:00   CM3 4UR             D       N   
3       12.506181  2019/1/11 0:00   SS0 9TY             T       N   
4       12.814481  2019/1/16 0:00  SS11 7BB             S       N   
...           ...             ...       ...           ...     ...   
691861  13.304687  2019/1/15 0:00  SW15 2TR             O       N   
691862  14.603968   2019/1/3 0:00  EC2A 4HB             O       N   
691863  13.473022   2019/2/1 0:00   SW9 9JB             T       N   
691864  14.058458  2019/1/25 0:00    E8 4AU             O       N   
691865  14.077876  2019/2/22 0:00  SW12 0BH             T       N   

       Property Rights       First address               Street  \
0                    F    FIR TREE COTTAGE           NORTH HILL   
1                    F               

C:\My_data\Work\Anaconda\envs\torch-gpu\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


Postcode
AL1 1AJ    0.500000
AL1 1AW    0.500000
AL1 1BX    0.731059
AL1 1DG    0.500000
AL1 1HF    0.500000
             ...   
YO8 9TU    0.500000
YO8 9XF    0.731059
YO8 9XG    0.500000
YO8 9XP    0.268941
YO8 9YE    0.731059
Name: beta, Length: 361469, dtype: float64
(553493,) (138373,)


C:\My_data\Work\Anaconda\envs\torch-gpu\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


Postcode
AL1 1AJ    0.731059
AL1 1BX    0.731059
AL1 1DU    0.731059
AL1 1HF    0.500000
AL1 1HY    0.731059
             ...   
YO8 9XF    0.731059
YO8 9XG    0.500000
YO8 9XL    0.731059
YO8 9XP    0.731059
YO8 9YE    0.731059
Name: beta, Length: 362819, dtype: float64
(553493,) (138373,)


...

County
BATH AND NORTH EAST SOMERSET    0.0
BEDFORD                         0.0
BLACKBURN WITH DARWEN           0.0
BLACKPOOL                       0.0
BLAENAU GWENT                   0.0
                               ... 
WOKINGHAM                       0.0
WORCESTERSHIRE                  0.0
WREKIN                          0.0
WREXHAM                         0.0
YORK                            0.0
Name: beta, Length: 113, dtype: float64
(553493,) (138373,)


C:\My_data\Work\Anaconda\envs\torch-gpu\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


County
BATH AND NORTH EAST SOMERSET    0.0
BEDFORD                         0.0
BLACKBURN WITH DARWEN           0.0
BLACKPOOL                       0.0
BLAENAU GWENT                   0.0
                               ... 
WOKINGHAM                       0.0
WORCESTERSHIRE                  0.0
WREKIN                          0.0
WREXHAM                         0.0
YORK                            0.0
Name: beta, Length: 113, dtype: float64
(553493,) (138373,)


C:\My_data\Work\Anaconda\envs\torch-gpu\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


County
BATH AND NORTH EAST SOMERSET     0.000000e+00
BEDFORD                          0.000000e+00
BLACKBURN WITH DARWEN            0.000000e+00
BLACKPOOL                        0.000000e+00
BLAENAU GWENT                   3.735334e-239
                                    ...      
WOKINGHAM                        0.000000e+00
WORCESTERSHIRE                   0.000000e+00
WREKIN                           0.000000e+00
WREXHAM                          0.000000e+00
YORK                             0.000000e+00
Name: beta, Length: 113, dtype: float64


In [12]:
from sklearn.preprocessing import LabelEncoder

# 创建LabelEncoder对象
le = LabelEncoder()
features['Property Type'] = le.fit_transform(features['Property Type'])
features['Old/New'] = le.fit_transform(features['Old/New'])
features['Property Rights'] = le.fit_transform(features['Property Rights'])

In [13]:
features = pd.get_dummies(features, columns=['Category Type'])
features['Category Type_A'] = features['Category Type_A'].astype('int8')
features['Category Type_B'] = features['Category Type_B'].astype('int8')
features = pd.get_dummies(features, columns=['Season'])
features['Season_Spring'] = features['Season_Spring'].astype('int8')
features['Season_Summer'] = features['Season_Summer'].astype('int8')
features['Season_Fall'] = features['Season_Fall'].astype('int8')
features['Season_Winter'] = features['Season_Winter'].astype('int8')

In [14]:
train = features[:train_counts]
test = features[train_counts:]

In [15]:
X_train = train
y_train = train_Price

In [16]:
from sklearn.model_selection import cross_val_predict
#采用十折交叉验证
n_folds = 10

def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=20)
    preds = cross_val_predict(model, X_train, y_train, cv=kf)
    rmse = np.sqrt(mean_squared_error(y_train, preds))
    return (rmse)

In [17]:
from bayes_opt import BayesianOptimization
from bayes_opt.util import UtilityFunction

def lgbm_cv(learning_rate, n_estimators, max_depth, num_leaves, subsample, colsample_bytree, min_child_weight, min_child_samples, reg_alpha, reg_lambda):
    lgbr_params = {'learning_rate': max(min(learning_rate, 1), 0),
                   'n_estimators': int(n_estimators),
                   'max_depth': int(max_depth),
                   'num_leaves': int(num_leaves),
                   'subsample': max(min(subsample, 1), 0),
                   'colsample_bytree': max(min(colsample_bytree, 1), 0),
                   'min_child_weight': max(min_child_weight, 0),
                   'min_child_samples': int(min_child_samples),
                   'random_state': 42,
                   'reg_alpha': max(reg_alpha, 0),
                   'boosting_type': 'gbdt',
                   'histogram_pool_size': 2048,
                   'reg_lambda': max(reg_lambda, 0)}
    lgbr = LGBMRegressor(**lgbr_params)
    return -np.mean(cross_val_score(lgbr, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))

bounds = {'learning_rate': (0.001, 0.1),
          'n_estimators': (500, 10000),
          'max_depth': (3, 15),
          'num_leaves': (20, 100),
          'subsample': (0.5, 1),
          'colsample_bytree': (0.5, 1),
          'min_child_weight': (0.001, 10),
          'min_child_samples': (10, 100),
          'reg_alpha': (0.001, 0.1),
          'reg_lambda': (0.001, 0.1)}

lgbm_bo = BayesianOptimization(lgbm_cv, bounds)
utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.0)
lgbm_bo.set_gp_params(alpha=1e-5, n_restarts_optimizer=2)
lgbm_bo.maximize(init_points=20, n_iter=100, acquisition_function=utility)

for i, res in enumerate(lgbm_bo.res):
    print("Iteration {}: \n\t{}".format(i, res))

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.08781   | 0.944     | 0.08166   | 3.439     | 88.68     | 1.814     | 6.227e+03 | 50.82     | 0.0297    | 0.07578   | 0.5063    |
| 2         | 0.08232   | 0.7589    | 0.02996   | 12.24     | 80.45     | 3.446     | 4.053e+03 | 59.69     | 0.04044   | 0.01713   | 0.6142    |
| 3         | 0.08347   | 0.9122    | 0.06042   | 10.29     | 29.12     | 5.112     | 9.647e+03 | 37.82     | 0.07855   | 0.04999   | 0.9995    |
| 4         | 0.08355   | 0.6605    | 0.01265   | 14.11     | 88.61     | 7.509     | 3.354e+03 | 64.18     | 0.03349   | 0.06822   | 0.611     |
| 5         | 0.0821    | 0.6095    | 0.02411   | 11.39     | 45.4      | 9.724     | 6.556e+03 | 75.21     | 0.04153   | 0.

| 34        | 0.0882    | 0.7716    | 0.08256   | 3.566     | 66.97     | 9.623     | 5.737e+03 | 89.34     | 0.03113   | 0.008089  | 0.7741    |
| 35        | 0.08322   | 0.9899    | 0.0748    | 8.922     | 64.95     | 6.33      | 1.764e+03 | 71.78     | 0.06151   | 0.01077   | 0.9815    |
| 36        | 0.09377   | 0.6902    | 0.03911   | 3.319     | 98.3      | 9.349     | 3.763e+03 | 92.11     | 0.07221   | 0.03156   | 0.9112    |
| 37        | 0.08306   | 0.5256    | 0.04533   | 8.505     | 96.41     | 5.748     | 3.758e+03 | 69.71     | 0.02934   | 0.02612   | 0.7989    |
| 38        | 0.08849   | 0.8496    | 0.07327   | 3.324     | 61.09     | 0.6135    | 5.743e+03 | 87.05     | 0.03928   | 0.0442    | 0.5279    |
| 39        | 0.09177   | 0.8796    | 0.007851  | 4.335     | 63.5      | 7.663     | 5.727e+03 | 82.15     | 0.0923    | 0.07586   | 0.9671    |
| 40        | 0.08843   | 0.6346    | 0.05868   | 4.398     | 68.99     | 6.997     | 1.772e+03 | 65.39     | 0.07896   | 0.

| 68        | 0.08342   | 0.6559    | 0.05918   | 8.599     | 72.15     | 0.6512    | 1.766e+03 | 56.51     | 0.09857   | 0.06225   | 0.5273    |
| 69        | 0.09549   | 0.5748    | 0.01926   | 4.025     | 67.04     | 6.883     | 1.758e+03 | 63.38     | 0.05428   | 0.0758    | 0.5148    |
| 70        | 0.1006    | 0.5615    | 0.03772   | 3.884     | 84.39     | 5.041     | 1.756e+03 | 66.0      | 0.03221   | 0.0148    | 0.5047    |
| 71        | 0.1078    | 0.7917    | 0.0047    | 4.929     | 82.12     | 2.552     | 1.756e+03 | 75.02     | 0.06382   | 0.08124   | 0.5284    |
| 72        | 0.1121    | 0.6077    | 0.01135   | 3.495     | 67.97     | 2.494     | 1.759e+03 | 68.52     | 0.04808   | 0.05443   | 0.6376    |
| 73        | 0.08386   | 0.9683    | 0.06914   | 6.376     | 73.69     | 1.37      | 1.757e+03 | 64.88     | 0.06395   | 0.00879   | 0.8996    |
| 74        | 0.08397   | 0.8771    | 0.08733   | 6.236     | 79.85     | 6.074     | 1.752e+03 | 67.83     | 0.07502   | 0.

| 102       | 0.09731   | 0.5149    | 0.05608   | 3.665     | 48.15     | 3.83      | 1.748e+03 | 92.47     | 0.008095  | 0.06249   | 0.8877    |
| 103       | 0.1688    | 0.7956    | 0.001343  | 3.371     | 40.73     | 9.029     | 1.75e+03  | 94.66     | 0.07258   | 0.02303   | 0.9124    |
| 104       | 0.08508   | 0.6668    | 0.06675   | 5.796     | 28.88     | 2.342     | 1.746e+03 | 89.39     | 0.05557   | 0.06964   | 0.8224    |
| 105       | 0.08255   | 0.9183    | 0.07633   | 12.87     | 27.55     | 8.666     | 1.759e+03 | 92.47     | 0.007287  | 0.04543   | 0.9737    |
| 106       | 0.09323   | 0.6256    | 0.01011   | 5.952     | 36.94     | 3.88      | 1.745e+03 | 97.72     | 0.09762   | 0.07474   | 0.9671    |
| 107       | 0.1051    | 0.7627    | 0.02163   | 3.473     | 33.24     | 4.719     | 1.76e+03  | 95.68     | 0.0971    | 0.0545    | 0.9532    |
| 108       | 0.08482   | 0.5105    | 0.01576   | 8.57      | 53.38     | 6.409     | 1.746e+03 | 99.28     | 0.0314    | 0.

In [18]:
print(lgbm_bo.max)

{'target': 0.16884613206171173, 'params': {'colsample_bytree': 0.7956323429425027, 'learning_rate': 0.0013430386136725034, 'max_depth': 3.371402491829888, 'min_child_samples': 40.729438079056045, 'min_child_weight': 9.029464802236694, 'n_estimators': 1750.2455826704856, 'num_leaves': 94.6577621427922, 'reg_alpha': 0.07258265630291426, 'reg_lambda': 0.02303351392898256, 'subsample': 0.9123720833111006}}


In [19]:
best_params = lgbm_bo.max['params']
best_params

{'colsample_bytree': 0.7956323429425027,
 'learning_rate': 0.0013430386136725034,
 'max_depth': 3.371402491829888,
 'min_child_samples': 40.729438079056045,
 'min_child_weight': 9.029464802236694,
 'n_estimators': 1750.2455826704856,
 'num_leaves': 94.6577621427922,
 'reg_alpha': 0.07258265630291426,
 'reg_lambda': 0.02303351392898256,
 'subsample': 0.9123720833111006}